In [1]:
import os
file_name="paper.pdf"

In [2]:
model="smollm"
from langchain_ollama import OllamaLLM
model=OllamaLLM(model=model)

In [3]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader(file_name)
pages=loader.load_and_split()
# print(f"the number of pages loaded from the document are: {len(pages)}\n\nContent of the first page:{'-'*20} \n\n{pages[0]}")

In [4]:
from langchain.prompts import PromptTemplate

template = """ Answer the question asked on the basis of the context provided.
If you can not find the answer then just say "answer is not available in the given text
context : {context}
Question : {question}
"""
prompt= PromptTemplate.from_template(template=template)
# print(prompt.format(context="Here is the context", question="Here is the question"))


In [5]:
from langchain_ollama import OllamaEmbeddings
embeddings=OllamaEmbeddings(model="smollm")

In [ ]:

from langchain_community.vectorstores import DocArrayInMemorySearch
vector_store=DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [7]:
retriever=vector_store.as_retriever()
# retriever.invoke("What is the result of the study")

In [8]:
from operator import itemgetter

chain = ({
    "context":itemgetter("question") | retriever,
    "question":itemgetter("question")
    }|
    prompt|model
    )

In [ ]:
question="What is the pdf about?"
while question !="":
    question = input("Enter your Question:...\n")
    print(chain.invoke({"question":question}))

In [16]:
# questions=[
#     "What is the title of the paper", 
#     "How many authors are there in the paper",
#     "what is the plateform used to run the experiment", 
#     "summerize the methodology"
# ]

# for question in questions:
#     print(f"question: {question}")
#     print(f"Answer:{chain.invoke({'question':question})}")
#     print()